# Project: Part-of-speech tagging with HMMs


## Inspect Data Provided as Part of SemEval 2017 Task 10
Need to format incoming data to be processed by NLTK

In [2]:
import unittest
import numpy as np

import pos
import pos_test
import nltk

# Load the Penn Treebank Corpus which will serve as our training set.
corpus = nltk.corpus.treebank
print 'There are %d sentences in the corpus.' % len(corpus.tagged_sents())
print 'The first sentence is:'
corpus.tagged_sents()[0]

There are 3914 sentences in the corpus.
The first sentence is:


[(u'Pierre', u'NNP'),
 (u'Vinken', u'NNP'),
 (u',', u','),
 (u'61', u'CD'),
 (u'years', u'NNS'),
 (u'old', u'JJ'),
 (u',', u','),
 (u'will', u'MD'),
 (u'join', u'VB'),
 (u'the', u'DT'),
 (u'board', u'NN'),
 (u'as', u'IN'),
 (u'a', u'DT'),
 (u'nonexecutive', u'JJ'),
 (u'director', u'NN'),
 (u'Nov.', u'NNP'),
 (u'29', u'CD'),
 (u'.', u'.')]

Load training and test date into the appropriate format

In [6]:
import codecs
corpus=[]
current_sent=[]
last_token=''
err='none'
with codecs.open("data/train.txt", 'r', 'utf-8') as f:

    for line in f.readlines():
        if not line in [u'\n', u'\r\n']:
            token, tag = line.split(' ')
            if last_token.endswith('.') and not last_token in ['e.g.', 'i.e.']:
                if token[0].isupper():
                    corpus.append(current_sent)
                    current_sent=[]
            current_sent.append((token, tag.strip()))
        else:
            corpus.append(current_sent)
            current_sent=[]
    corpus.append(current_sent)
    
with codecs.open("data/dev.txt", 'r', 'utf-8') as f:

    for line in f.readlines():
        if not line in [u'\n', u'\r\n']:
            token, tag = line.split(' ')
            if last_token.endswith('.') and not last_token in ['e.g.', 'i.e.']:
                if token[0].isupper():
                    corpus.append(current_sent)
                    current_sent=[]
            current_sent.append((token, tag.strip()))
        else:
            corpus.append(current_sent)
            current_sent=[]
    corpus.append(current_sent)

In [7]:
import os
import pos
import nltk

y_pred = []
y_expected = []

hmm = pos.HMM()
for sentence in corpus:
    hmm.update_counts(sentence)
hmm.compute_logprobs()
def pretty_print_fb(sentence):
    #print sentence
    #print hmm.forward_backward(sentence.split())
    return hmm.forward_backward(sentence.split())

#for x in xrange(1):

corpus=[]
current_sent=[]
last_token=''
err='none'
# with codecs.open("train.txt", 'r', 'utf-8') as f:
with codecs.open("data/test.txt", 'r', 'utf-8') as f:

    for line in f.readlines():
        if not line in [u'\n', u'\r\n']:
            token, tag = line.split(' ')
            if last_token.endswith('.') and not last_token in ['e.g.', 'i.e.']:
                if token[0].isupper():
                    corpus.append(current_sent)
                    current_sent=[]
            current_sent.append((token, tag.strip()))
        else:
            corpus.append(current_sent)
            current_sent=[]
    corpus.append(current_sent)

for x in xrange(len(corpus)):
    
    #print("Predicted:")
    #y_pred = pretty_print_fb(' '.join([i for i,j in corpus[x]]))
    y_pred_result = pretty_print_fb(' '.join([i for i,j in corpus[x]]))
    y_pred = np.append(y_pred, [y_pred_result])
    
    #print("Expected:")
    #y_expected = [j for i, j in corpus[x]]
    y_expected_result = [j for i, j in corpus[x]]
    y_expected = np.append(y_expected, [y_expected_result])

num_pos_predictions = len([i for i in y_pred if i != "O"])
num_pos_predictions
num_true_labels = len([i for i in y_expected if i != "O"])
num_true_labels

print("Positive Preditions: ", num_pos_predictions, " Number of True Labels: ", num_true_labels)

num_correct = 0
num_correct_all = 0
for (a,b) in zip(y_pred, y_expected):
    if (a == b):
        num_correct_all += 1
        if (a != "O"):
            num_correct += 1



    
precision = num_correct * 1.0 / num_pos_predictions
recall = num_correct * 1.0 / num_true_labels
accuracy = num_correct_all * 1.0 / len(y_pred)
f1 = 2 * precision * recall / (precision + recall)
print "Num_Correct: ", num_correct
print "Num_Pos_Predictions: ", num_pos_predictions
print "Precision: ", precision
print "Accuracy: ", accuracy
print "F1: ", f1



('Positive Preditions: ', 42, ' Number of True Labels: ', 4893)
Num_Correct:  5
Num_Pos_Predictions:  42
Precision:  0.119047619048
Accuracy:  0.866705502304
F1:  0.00202634245187


In [281]:
#y_pred.count('B-Material')
#l.count('b')
#print recall

#>>> a = numpy.array([0, 3, 0, 1, 0, 1, 2, 1, 0, 0, 0, 0, 1, 3, 4])
unique, counts = np.unique(y_expected, return_counts=True)
dict(zip(unique, counts))

{u'B-Material': 2579,
 u'B-Process': 2959,
 u'B-Task': 1127,
 u'I-Material': 2816,
 u'I-Process': 5887,
 u'I-Task': 4281,
 u'O': 86920}

In [22]:
import numpy as np
y_expected_array = np.asarray(y_expected)
print y_expected_array.shape
y_pred_array = np.asarray(y_pred)
print y_pred_array.shape


(605,)
(605,)


Before continuing, let's take a few moments to inspect the format of the training data.  The Treebank Reader object has a `tagged_sents()` function that returns an list of sentences.  Each sentence consists of a list of (word, part of speech) tuples.

In [8]:
test_sentence=[(u'A', u'O'),
 (u'hydroxyl-functionalized', u'O'),
 (u'poly(butylene', u'O'),
 (u'succinate)', u'O'),
 (u'based', u'O'),
 (u'polyester', u'B-Material'),
 (u'was', u'O'),
 (u'prepared', u'O'),
 (u'by', u'O'),
 (u'conventional', u'O'),
 (u'polycondensation', u'B-Process'),
 (u'of', u'O'),
 (u'benzyl-protected', u'O'),
 (u'dimethyl', u'B-Material'),
 (u'malonate', u'I-Material'),
 (u'and', u'O'),
 (u'1,4-butanediol', u'B-Material'),
 (u'(Scheme', u'O'),
 (u'2(a))', u'O'),
 (u'[24a].', u'O'),
 (u'Yao', u'O'),
 (u'et\xa0al.', u'O'),
 (u'reported', u'O'),
 (u'on', u'O'),
 (u'the', u'O'),
 (u'direct', u'O'),
 (u'polycondensation', u'O'),
 (u'of', u'O'),
 (u'l-lactic', u'B-Material'),
 (u'acid', u'I-Material'),
 (u'and', u'O'),
 (u'citric', u'O'),
 (u'acid', u'O'),
 (u'with', u'O'),
 (u'the', u'O'),
 (u'formation', u'O'),
 (u'of', u'O'),
 (u'poly[(l-lactic', u'O'),
 (u'acid)-co-(citric', u'O'),
 (u'acid)],', u'O'),
 (u'obtaining', u'O'),
 (u'a', u'O'),
 (u'polyester', u'B-Material'),
 (u'oligomer', u'I-Material'),
 (u'with', u'O'),
 (u'both', u'O'),
 (u'pendant', u'O'),
 (u'carboxylic', u'O'),
 (u'and', u'O'),
 (u'hydroxyl', u'O'),
 (u'groups', u'O'),
 (u'[24b].', u'O'),
 (u'This', u'O'),
 (u'PLCA', u'B-Material'),
 (u'oligomer', u'I-Material'),
 (u'was', u'O'),
 (u'reacted', u'O'),
 (u'with', u'O'),
 (u'dihydroxylated', u'B-Material'),
 (u'PLLA', u'I-Material'),
 (u'as', u'O'),
 (u'a', u'O'),
 (u'macromonomer,', u'O'),
 (u'yielding', u'O'),
 (u'a', u'O'),
 (u'PLCA\u2013PLLA', u'B-Material'),
 (u'multiblock', u'I-Material'),
 (u'copolymer', u'I-Material'),
 (u'as', u'I-Material'),
 (u'shown', u'I-Material'),
 (u'in', u'I-Material'),
 (u'Scheme', u'O'),
 (u'2(b).', u'O'),
 (u'While', u'O'),
 (u'lipases', u'O'),
 (u'have', u'O'),
 (u'been', u'O'),
 (u'investigated', u'O'),
 (u'for', u'O'),
 (u'the', u'O'),
 (u'ring-opening', u'O'),
 (u'polymerization', u'O'),
 (u'(ROP)', u'O'),
 (u'of', u'O'),
 (u'cyclic', u'B-Material'),
 (u'ester', u'B-Process'),
 (u'monomers', u'I-Process'),
 (u'[25,26],', u'O'),
 (u'they', u'B-Process'),
 (u'have', u'O'),
 (u'also', u'O'),
 (u'been', u'B-Material'),
 (u'used', u'I-Material'),
 (u'for', u'I-Material'),
 (u'the', u'O'),
 (u'preparation', u'O'),
 (u'of', u'O'),
 (u'polyesters', u'O'),
 (u'by', u'O'),
 (u'polycondensation', u'O'),
 (u'reactions.', u'O'),
 (u'The', u'O'),
 (u'advantage', u'O'),
 (u'of', u'O'),
 (u'this', u'O'),
 (u'technique', u'O'),
 (u'is', u'B-Process'),
 (u'that', u'O'),
 (u'these', u'O'),
 (u'enzyme-catalyzed', u'O'),
 (u'reactions', u'O'),
 (u'proceed', u'O'),
 (u'without', u'O'),
 (u'protection', u'O'),
 (u'of', u'O'),
 (u'the', u'O'),
 (u'pendant', u'B-Process'),
 (u'functional', u'O'),
 (u'groups.', u'O'),
 (u'In', u'O'),
 (u'this', u'O'),
 (u'field,', u'O'),
 (u'hydroxyl-bearing', u'O'),
 (u'polyesters', u'O'),
 (u'have', u'O'),
 (u'been', u'O'),
 (u'synthesized', u'O'),
 (u'by', u'O'),
 (u'the', u'O'),
 (u'copolymerization', u'B-Material'),
 (u'of', u'I-Material'),
 (u'divinyl', u'O'),
 (u'adipate', u'O'),
 (u'with', u'O'),
 (u'various', u'O'),
 (u'triols', u'O'),
 (u'(e.g.', u'B-Process'),
 (u'glycerol,', u'O'),
 (u'1,2,4-butanetriol)', u'B-Material'),
 (u'as', u'I-Material'),
 (u'represented', u'O'),
 (u'in', u'O'),
 (u'Scheme', u'O'),
 (u'2(c)', u'O'),
 (u'[27]', u'O'),
 (u'and', u'O'),
 (u'by', u'O'),
 (u'copolymerizations', u'O'),
 (u'of', u'O'),
 (u'1,8-octanediol', u'O'),
 (u'with', u'O'),
 (u'adipic', u'O'),
 (u'acid', u'O'),
 (u'and', u'O'),
 (u'several', u'B-Process'),
 (u'alditols', u'O'),
 (u'[28].', u'B-Material'),
 (u'Very', u'O'),
 (u'recently,', u'O'),
 (u'several', u'O'),
 (u'\u03b1-hydroxy', u'O'),
 (u'acids', u'O'),
 (u'derived', u'O'),
 (u'from', u'O'),
 (u'amino', u'O'),
 (u'acids', u'O'),
 (u'were', u'O'),
 (u'homo-', u'I-Material'),
 (u'and', u'I-Material'),
 (u'copolymerized', u'O'),
 (u'with', u'O'),
 (u'lactic', u'O'),
 (u'acid', u'O'),
 (u'by', u'O'),
 (u'polycondensation', u'O'),
 (u'in', u'O'),
 (u'bulk', u'O'),
 (u'without', u'O'),
 (u'protected', u'O'),
 (u'monomers', u'O'),
 (u'(Scheme', u'O'),
 (u'2(d))', u'O'),
 (u'[29].', u'O'),
 (u'Biodegradable', u'O'),
 (u'polyesters', u'O'),
 (u'with', u'O'),
 (u'various', u'O'),
 (u'pendant', u'B-Process'),
 (u'groups', u'O'),
 (u'were', u'B-Material'),
 (u'obtained,', u'I-Material'),
 (u'although', u'B-Material'),
 (u'the', u'I-Material'),
 (u'molecular', u'I-Material'),
 (u'weights', u'O'),
 (u'remained', u'O'),
 (u'low', u'O'),
 (u'(1000\u20133000gmol\u22121).', u'O')]

In [20]:
import os
import pos
import nltk

y_pred2 = []
y_expected2 = []

y_pred_result2 = pretty_print_fb(' '.join([i for i,j in test_sentence]))
print("Predicted:")
print y_pred_result2
y_pred2 = np.append(y_pred2, [y_pred_result2])

print("Expected:")
#y_expected = [j for i, j in corpus[x]]
y_expected_result2 = [j for i,j in test_sentence]
print y_expected_result2
y_expected2 = np.append(y_expected2, [y_expected_result2])

num_pos_predictions = len([i for i in y_pred2 if i != "O"])
num_pos_predictions
num_true_labels = len([i for i in y_expected2 if i != "O"])
num_true_labels

print("Positive Preditions: ", num_pos_predictions, " Number of True Labels: ", num_true_labels)

num_correct = num_correct_all = 0
for (a,b) in zip(y_pred2, y_expected2):
    if (a == b):
        num_correct_all += 1
        if (a != "O"):
            num_correct += 1



    
precision = num_correct * 1.0 / num_pos_predictions
recall = num_correct * 1.0 / num_true_labels
accuracy = num_correct_all * 1.0 / len(y_pred2)
f1 = 2 * precision * recall / (precision + recall)
print "Num_Correct: ", num_correct
print "Num_Pos_Predictions: ", num_pos_predictions
print "Precision: ", precision
print "Accuracy: ", accuracy
print "F1: ", f1


Predicted:
[u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O'

ZeroDivisionError: float division by zero